<a href="https://colab.research.google.com/github/honeycaron/TIL/blob/main/IMDB_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98(Glove_%EC%82%AC%EC%9A%A9)_%EC%86%90%ED%97%8C_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import imdb

In [ ]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-12-10 00:54:26--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.79.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.79.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  69.5MB/s    in 22s     

2020-12-10 00:54:48 (72.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
import gensim

In [ ]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [ ]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))

In [ ]:
word_to_index = imdb.get_word_index()
index_to_word={}
for key, value in word_to_index.items():
    if value+3 < vocab_size:
        index_to_word[value+3] = key

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
len(index_to_word)

9996

In [ ]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [ ]:
np.shape(embedding_matrix)

(10000, 300)

In [ ]:
for i, word in index_to_word.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [ ]:
index_to_word.items()

dict_items([(1411, 'woods'), (2348, 'hanging'), (2292, 'woody'), (6749, 'arranged'), (2341, 'bringing'), (1639, 'wooden'), (4013, 'errors'), (3233, 'dialogs'), (362, 'kids'), (5037, 'uplifting'), (7096, 'controversy'), (9881, 'projection'), (7183, 'stern'), (5624, 'morally'), (5286, 'wang'), (181, 'want'), (2106, 'travel'), (6705, 'barbra'), (3933, 'dinosaurs'), (355, 'wrong'), (4763, 'subplots'), (9095, 'welcomed'), (6706, 'butcher'), (1183, 'fit'), (1930, 'screaming'), (4290, 'fix'), (9657, 'hurting'), (302, 'effects'), (8778, 'barton'), (6195, 'ingrid'), (7919, 'adapt'), (4014, 'disturbed'), (1288, 'purpose'), (6795, 'olds'), (887, 'needed'), (1300, 'master'), (5402, 'positively'), (9839, 'zatoichi'), (547, 'feeling'), (5625, 'affairs'), (7802, 'wholesome'), (1360, 'cinematic'), (4991, 'tech'), (660, 'saying'), (8425, 'padded'), (5748, 'tempted'), (7479, 'plate'), (3902, 'altogether'), (8779, 'lds'), (1780, 'nicely'), (4591, 'mummy'), (775, 'lots'), (9262, 'lotr'), (9882, 'irs'), (6

In [ ]:
# word2vec_model['woods']

In [ ]:
# embedding_matrix[1411]

In [ ]:
embedding_matrix[1411]==word2vec_model['woods']  #임베딩 잘 됐는지 체크

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
# 임베딩 안된 행의 갯수 확인
i=0
for n in embedding_matrix:
  if sum(n)==0:
    i+=1
  
print(i)


718


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [ ]:
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
# model.add(Flatten())
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
333/334 [============================>.] - ETA: 0s - loss: 0.5379 - acc: 0.7341
Epoch 00001: val_acc improved from -inf to 0.80120, saving model to GRU_model.h5
334/334 [==============================] - 12s 37ms/step - loss: 0.5377 - acc: 0.7342 - val_loss: 0.4586 - val_acc: 0.8012
Epoch 2/15
333/334 [============================>.] - ETA: 0s - loss: 0.3792 - acc: 0.8380
Epoch 00002: val_acc improved from 0.80120 to 0.86520, saving model to GRU_model.h5
334/334 [==============================] - 12s 36ms/step - loss: 0.3791 - acc: 0.8381 - val_loss: 0.3187 - val_acc: 0.8652
Epoch 3/15
333/334 [============================>.] - ETA: 0s - loss: 0.3189 - acc: 0.8651
Epoch 00003: val_acc improved from 0.86520 to 0.87860, saving model to GRU_model.h5
334/334 [==============================] - 12s 36ms/step - loss: 0.3187 - acc: 0.8652 - val_loss: 0.2893 - val_acc: 0.8786
Epoch 4/15
333/334 [============================>.] - ETA: 0s - loss: 0.2848 - acc: 0.8839
Epoch 00004: val_a

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 11s 13ms/step - loss: 0.2579 - acc: 0.8932

 테스트 정확도: 0.8932
